# <center>Machine Learning in Python</center> 

Mount your Google drive by using this code snippet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In this exercise, we will build a neural network model from scratch that is able to distinguish dogs from cats. 

**NOTE:** The 2200 images used in this exercise are excerpted from the ["Dogs vs. Cats"](https://www.kaggle.com/c/dogs-vs-cats/data) dataset available on Kaggle, which contains 25,000 images. We use a subset of the full dataset here.


#### 1.1 Load dataset
 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 128, 128

train_data_dir = '/content/drive/My Drive/Workshop-Keras/dataset/dogs-vs-cats/train/'
validation_data_dir = '/content/drive/My Drive/Workshop-Keras/dataset/dogs-vs-cats/val/' 
test_data_dir = '/content/drive/My Drive/Workshop-Keras/dataset/dogs-vs-cats/'

# normalize the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
batch_size = 32 # e.g 4, 8, 16, 32, 64

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        color_mode = 'grayscale',
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        color_mode = 'grayscale',
        batch_size=batch_size,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        color_mode = 'grayscale',
        classes = ['test'],
        class_mode=None)

Using TensorFlow backend.


In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.optimizers import SGD

img_width, img_height = 50, 50
input_dim = 50*50
output_dim = 1

def CNN():
    model = Sequential()
    model.add(Conv2D(32, (3,3), input_shape = (img_width, img_height, 3), padding='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Flatten())
    model.add(Dense(500, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid')) 
    return model

# define model
model = CNN()
# define optimizer
sgd = SGD(lr=0.001)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# print model information
model.summary()


nb_epoch = 100
nb_train_samples = 2000
nb_validation_samples = 200

import time
start = time.time()
model_info = model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)
end = time.time()
print("Model took %0.2f seconds to train"%(end - start))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # model history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['training acc', 'val acc'], loc='best')
    axs[0].grid(True)
    # model history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['training loss', 'val loss'], loc='best')
    axs[1].grid(True)
    plt.show()
plot_model_history(model_info)